In [1]:
from wallcheb.qtmlib.circuits.lcu import LCUMultiplexorBox

In [2]:
from wallcheb.operators import ising_model

n_state_qubits = 4
ham = ising_model(n_qubits=n_state_qubits, j=1.0, h=0.5)

In [3]:
from pytket.circuit.display import render_circuit_jupyter

multiplexor_lcu = LCUMultiplexorBox(ham, n_state_qubits)
n_prep_qubits = multiplexor_lcu.n_prepare_qubits

In [4]:
from pytket.passes import AutoRebase
from pytket import OpType
from pytket.passes import DecomposeBoxes

circ = multiplexor_lcu.get_circuit()
DecomposeBoxes().apply(circ)
rebase = AutoRebase({OpType.CX, OpType.Rz, OpType.H, OpType.CCX})
rebase.apply(circ)

from guppylang import guppy

qlibs_multiplexor_lcu = guppy.load_pytket("qlibs_multiplexor_lcu", circ)

In [5]:
print(qlibs_multiplexor_lcu)

GuppyDefinition(wrapped=RawLoadPytketDef(id=DefId(id=309), name='qlibs_multiplexor_lcu', defined_at=None, source_span=None, input_circuit=[Rz(3.5) p[0]; Rz(3.5) p[1]; Rz(3.5) p[2]; Rz(3) q[0]; H q[1]; H q[2]; H q[3]; H p[0]; H p[1]; H p[2]; H q[0]; Rz(3.125) q[1]; Rz(2.875) q[2]; Rz(0.875) q[3]; Rz(0.36901) p[0]; Rz(0.423099) p[1]; Rz(0.347957) p[2]; Rz(2.875) q[0]; H p[0]; H p[1]; H p[2]; Rz(0.5) p[0]; Rz(0.5) p[1]; Rz(0.5) p[2]; CX p[0], p[1]; Rz(3.5) p[1]; H p[1]; Rz(0.0312722) p[1]; H p[1]; Rz(0.5) p[1]; CX p[0], p[1]; CX p[1], p[2]; Rz(3.5) p[2]; H p[2]; Rz(0.152043) p[2]; H p[2]; Rz(0.5) p[2]; CX p[0], p[2]; Rz(3.5) p[2]; H p[2]; Rz(3.90204) p[2]; H p[2]; Rz(0.5) p[2]; CX p[1], p[2]; CX p[1], q[2]; Rz(3.5) p[2]; H p[2]; Rz(0.875) q[2]; Rz(0.0979566) p[2]; H q[2]; H p[2]; Rz(0.375) q[2]; Rz(0.5) p[2]; H q[2]; CX p[0], p[2]; CX p[0], q[1]; CX p[2], q[0]; CX p[0], q[2]; CX p[2], q[3]; Rz(3.125) q[0]; Rz(2.375) q[1]; H q[0]; H q[1]; H q[2]; Rz(0.875) q[3]; Rz(0.875) q[0]; Rz(3.5) q[1

In [6]:
from guppylang.std.builtins import comptime
from guppylang.std.quantum import qubit, discard_array, measure, measure_array
from hugr.qsystem.result import QsysResult
from guppylang.std.builtins import result, array, exit
from typing import Callable

@guppy
def power_block_encoding(power:int, block_encoding: Callable[[array[qubit, comptime(n_prep_qubits)], array[qubit, comptime(n_state_qubits)]], None], state_qreg: array[qubit,comptime(n_state_qubits)]) -> None:

    for i in range(power):

        prep_qreg = array(qubit() for _ in range(comptime(n_prep_qubits)))
        block_encoding(prep_qreg, state_qreg)
        
        outcome = measure_array(prep_qreg)
        
        for b in outcome: 
            if b:
                exit("circuit failed",1)


In [7]:
@guppy
def main() -> None:
    """Main function to run the multiplexor LCU circuit."""
    state_qreg = array(qubit() for _ in range(comptime(n_state_qubits)))

    block_encoding = qlibs_multiplexor_lcu
    power = 7

    power_block_encoding(power, block_encoding, state_qreg)

    result('c',measure_array(state_qreg))

compiled_hugr = guppy.compile(main)


In [8]:
from hugr.qsystem.result import QsysResult
from selene_sim import build, Quest

n_shots = 80000
runner = build(compiled_hugr)
shots = QsysResult(
    runner.run_shots(
        Quest(random_seed=2), n_qubits=multiplexor_lcu.n_qubits, n_shots=n_shots
    )
)

print(f'Final state:{shots.register_counts()["c"]}, when initial state is |000>')

shots_counts = shots.register_counts()["c"]
success_prob = sum(shots_counts.values())/ n_shots
print(f"Success probability (from circuits): {success_prob:.4f}")

Final state:Counter({'0000': 235, '1000': 19, '0001': 18, '0100': 7, '0010': 4, '0111': 1, '1001': 1, '1111': 1}), when initial state is |000>
Success probability (from circuits): 0.0036
